## Modelling

# Feature Engineering

We now engineer the dataset to prepare features for modeling.

## Prediction Targets

We need to predict:
- **WholesaleUnitPrice**
- **RetailUnitPrice** 

These will be treated as two separate targets.

## Feature Engineering Process

We will:

1. **Create lag features** (to include past prices as predictors)
2. **Encode categorical variables**
3. **Drop irrelevant columns**
4. **Split into feature matrix X and targets y_wholesale, y_retail**

## Lag Features

When working with time-series or sequential data, the current value of a variable often depends on its past values.

For example:
- The wholesale price of beans today may be influenced by the price yesterday
- The retail price of tomatoes this week may depend on the wholesale price last week

Lag features capture this dependency by adding past observations as input features for the current prediction.

### How it Works

A lag feature simply shifts the time-series backward by a specified number of time steps.

**Example:**

| Date       | Price |
|------------|-------|
| 2024-01-01 | 100   |
| 2024-01-02 | 110   |
| 2024-01-03 | 105   |

If we create a lag-1 feature (Price_t-1), the dataset becomes:

| Date       | Price | Price_t-1 |
|------------|-------|-----------|
| 2024-01-02 | 110   | 100       |
| 2024-01-03 | 105   | 110       |

### In This Project

We created two lag features:
- **Wholesale_t-1**: Previous wholesale price
- **Retail_t-1**: Previous retail price

These features help our models capture short-term dependencies in price movements, making predictions more accurate.

## Target Transformation

We will also apply **log transformation** to the targets since they are skewed. This helps:
- Normalize the distribution
- Reduce the impact of outliers
- Improve model performance

### BASELINE MODEL PART 1:RIDGE REGRESSION


We begin our modeling with **Ridge Regression** as a baseline. Ridge helps address multicollinearity and penalizes high-magnitude coefficients to prevent overfitting, especially important in datasets with many features or one-hot encodings.

### Data Preparation for Modeling

Before training our model, we perform several preprocessing steps:

- **Sorting and Lag Feature Engineering:**  
  To capture temporal trends, we sort the dataset by date and group, then create lagged features (`Wholesale_t-1` and `Retail_t-1`) to feed past price information into the model.

In [25]:

# Sort for lagging
df_cleaned = df_clean.sort_values(by=["Commodity", "Market", "County", "Date"])

# Lag features
df_cleaned["Wholesale_t-1"] = df_cleaned.groupby(["Commodity", "Market", "County"])["WholesaleUnitPrice"].shift(1)
df_cleaned["Retail_t-1"] = df_cleaned.groupby(["Commodity", "Market", "County"])["RetailUnitPrice"].shift(1)
df = df_cleaned.dropna()


### Outlier Removal, One-Hot Encoding, and Target Transformation
We continue preparing the dataset by:

- **Removing Extreme Outliers:**  
  Cap the prices at the 1st and 99th percentiles to reduce skew caused by extreme values.

- **One-Hot Encoding:**  
  Convert categorical features (like Commodity and Market) into binary features to make them usable in the regression model.

- **Log Transforming Targets:**  
  Apply `log1p` to the wholesale and retail prices to stabilize variance and handle skewed distributions.s

In [26]:
# Remove extreme outliers
low_cap = 1
high_cap_wholesale = df["WholesaleUnitPrice"].quantile(0.99)
high_cap_retail = df["RetailUnitPrice"].quantile(0.99)
df = df[(df["WholesaleUnitPrice"] >= low_cap) & (df["RetailUnitPrice"] >= low_cap)]
df = df[(df["WholesaleUnitPrice"] <= high_cap_wholesale) & (df["RetailUnitPrice"] <= high_cap_retail)]


# One-hot encode categoricals
categoricals = ["Commodity", "Classification", "County", "Market", "Month_name", "Day_Name"]
df_encoded = pd.get_dummies(df, columns=categoricals, drop_first=True)

# Targets (log-transformed for stability)
y_wholesale = np.log1p(df_encoded["WholesaleUnitPrice"])
y_retail = np.log1p(df_encoded["RetailUnitPrice"])

# Features 
drop_cols = ["Date", "WholesaleUnitPrice", "RetailUnitPrice", "ReferencePrice"]
X = df_encoded.drop(columns=drop_cols)
X_df = pd.DataFrame(X, index=df_encoded.index) 


### Train-Test Split & Feature Scaling

We split the data into training and testing sets (80/20) for both wholesale and retail price models.

**Scaling**

 Since Ridge regression is sensitive to feature magnitudes, we apply **standard scaling** (`StandardScaler`) after the split to avoid data leakage

In [27]:
#  Train-Test Split (with index tracking)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_df, y_wholesale, test_size=0.2, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_df, y_retail, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_w_scaled = scaler.fit_transform(X_train_w)
X_test_w_scaled = scaler.transform(X_test_w)
X_train_r_scaled = scaler.fit_transform(X_train_r)
X_test_r_scaled = scaler.transform(X_test_r)


### Hyperparameter Tuning with RidgeCV

We use **RidgeCV** to automatically select the best alpha (regularization strength) through cross-validation.

- A range of alpha values (log-spaced from 0.001 to 1000) is tested.
- The model fits the training data and selects the optimal value based on cross-validation performance.

In [28]:
#  RidgeCV (Hyperparameter tuning)
alphas = np.logspace(-3, 3, 50)

ridge_cv_wholesale = RidgeCV(alphas=alphas, cv=5)
ridge_cv_retail = RidgeCV(alphas=alphas, cv=5)

ridge_cv_wholesale.fit(X_train_w_scaled, y_train_w)
ridge_cv_retail.fit(X_train_r_scaled, y_train_r)

print(f"Best alpha (Wholesale): {ridge_cv_wholesale.alpha_}")
print(f"Best alpha (Retail): {ridge_cv_retail.alpha_}")

# 4. Predictions (back to KES scale)
y_pred_w = np.expm1(ridge_cv_wholesale.predict(X_test_w_scaled))
y_pred_r = np.expm1(ridge_cv_retail.predict(X_test_r_scaled))
y_test_w_orig = np.expm1(y_test_w)
y_test_r_orig = np.expm1(y_test_r)


Best alpha (Wholesale): 184.20699693267164
Best alpha (Retail): 59.636233165946365


### Error analysis by county and commodity

To gain a deeper understanding of our model’s prediction performance, we conduct an error analysis grouped by **Commodity** and **County**:

- Create a results DataFrame comparing actual vs. predicted values for both wholesale and retail prices.
- Join this with the original group labels (Commodity, County).
- Compute the **prediction error** as the difference between predicted and actual values.
- Aggregate the errors using `groupby()` to find the **average prediction error** per commodity and county.

This helps identify where our model systematically **overestimates** or **underestimates** prices, key for guiding model improvements and flagging specific market dynamics.

In [29]:
# Error Analysis (by Commodity & County)
results_df = pd.DataFrame({
    "Actual_Wholesale": y_test_w_orig,
    "Pred_Wholesale": y_pred_w,
    "Actual_Retail": y_test_r_orig,
    "Pred_Retail": y_pred_r
}, index=X_test_w.index)

# Join original group labels
results_df = results_df.join(df[["Commodity", "County"]])

# Compute errors
results_df["Wholesale_Error"] = results_df["Pred_Wholesale"] - results_df["Actual_Wholesale"]
results_df["Retail_Error"] = results_df["Pred_Retail"] - results_df["Actual_Retail"]

# Summarize by group
error_summary = results_df.groupby(["Commodity", "County"])[["Wholesale_Error", "Retail_Error"]].mean()


### Model Evaluation: Metrics
We assess model performance using:

- **Mean Squared Error (MSE)** – Measures average squared error.
- **R² Score** – Indicates the proportion of variance explained by the model.



In [30]:
# Metrics
mse_w = mean_squared_error(y_test_w_orig, y_pred_w)
mse_r = mean_squared_error(y_test_r_orig, y_pred_r)
r2_w = r2_score(y_test_w_orig, y_pred_w)
r2_r = r2_score(y_test_r_orig, y_pred_r)

print(f"Wholesale MSE: {mse_w:.2f}, R²: {r2_w:.4f}")
print(f"Retail MSE: {mse_r:.2f}, R²: {r2_r:.4f}")


Wholesale MSE: 500.84, R²: 0.7850
Retail MSE: 704.32, R²: 0.7790



These are strong baseline results, suggesting our model captures a significant portion of the price variation.


### Identifying Top Overestimations and Underestimations
We sort the grouped error values to reveal where the model's predictions deviate the most:

- **Overestimations:** Where predicted prices are significantly higher than actual prices.
- **Underestimations:** Where predicted prices are significantly lower than actual prices.

We display the **top 10** cases for both wholesale and retail price predictions.

These insights help pinpoint areas where price volatility, insufficient features, or market anomalies may be impacting model accuracy, and where further refinement (e.g., location-specific adjustments or additional features) may be neceary. 
essary.
ection).


In [31]:
# Sort and display top overestimations
print("\nTop 10 Overestimations (Wholesale):")
print(error_summary.sort_values(by="Wholesale_Error", ascending=False).head(10))

print("\nTop 10 Overestimations (Retail):")
print(error_summary.sort_values(by="Retail_Error", ascending=False).head(10))



Top 10 Overestimations (Wholesale):
                          Wholesale_Error  Retail_Error
Commodity   County                                     
Rice        Kisii               94.840987    286.435716
Beans       Garissa             88.050464     94.206223
Wheat       Siaya               60.398434    152.354299
Rice        Kilifi              59.096366     58.824800
Maize Flour Kakamega            48.502019     49.355801
            Nandi               43.011909     40.709170
Beans       Tana-River          32.043559     67.526660
Rice        Taita-Taveta        31.056484     14.931256
            Meru                25.479767     14.750780
            Tana-River          23.547252     16.912627

Top 10 Overestimations (Retail):
                        Wholesale_Error  Retail_Error
Commodity   County                                   
Rice        Kisii             94.840987    286.435716
Wheat       Siaya             60.398434    152.354299
Beans       Garissa           88.050464  

In [32]:

# Sort and display top underestimations
print("\nTop 10 Underestimations (Wholesale):")
print(error_summary.sort_values(by="Wholesale_Error", ascending=True).head(10))

print("\nTop 10 Underestimations (Retail):")
print(error_summary.sort_values(by="Retail_Error", ascending=True).head(10))



Top 10 Underestimations (Wholesale):
                          Wholesale_Error  Retail_Error
Commodity  County                                      
Tea        Uasin-Gishu         -45.316470    -46.573909
Avocado    Lamu                -41.620867    -42.042864
Wheat      Taita-Taveta        -41.093198    -25.526005
Tomatoes   Tana-River          -38.237687    -26.135669
Avocado    Tana-River          -32.896199     -7.623530
Carrots    Kisii               -29.563084    -23.039765
Avocado    Kisumu              -29.521318    -49.715618
Rice       Tharaka-Nithi       -27.952917    -25.125234
Dry Onions Nyamira             -27.265990    -38.688600
Tomatoes   Trans-Nzoia         -25.316067    -24.796633

Top 10 Underestimations (Retail):
                         Wholesale_Error  Retail_Error
Commodity   County                                    
Avocado     Kisumu            -29.521318    -49.715618
Tea         Uasin-Gishu       -45.316470    -46.573909
Mangoes     Laikipia           -0.5